In [2]:
#!pip install opencv-python

In [4]:
from tensorflow.keras.datasets import mnist    #載入MNIST Datasets
from tensorflow.keras.utils import to_categorical   #to_categorical是對label資料做one-hot encoding的method
# import util1 as u  #自己寫的類別裡面包含很多自訂函式可調出來應用

adj_size = 0   # 指定要將數字調整為多少像素, 設為 0 表示不調整
fns = ''        # 檔名附加訊息

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape),print(train_labels.shape)  #(60000, 28, 28),(60000,) 六萬張訓練集,像素為28*28
print(test_images.shape),print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


(None, None)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2          # 匯入 OpenCV 影像處理套件
import glob         # 匯入內建的檔案與資料夾查詢套件

In [6]:
def img_best(img, size=20, vdif=1, hdif=1):
    if size <= 0: return img

    # 1.先將影像上下左右置中
    img = img_center(img, vdif, hdif)

    # 2.調到 上+下高度 == 8
    a, b, c, d = img_getBorder(img)
    border = 28-size
    if a+b < border:  # 空太少, 圖太大, 要縮小
        img = img_bigger(img, -(border-(a+b)+2))
    elif a+b > border and c+d > border: # 空太大, 圖太小, 要放大
        img = img_bigger(img, min(a+b-border, c+d-border))

    # 3.再次將影像上下左右置中
    img = img_center(img, vdif, hdif)

    return img

In [11]:
if adj_size > 0:
    fns = '_S' + str(adj_size)
    print(f'調整 MNIST 圖片的數字大小改為 {adj_size} 像素, 並置中')
    for i in range(len(train_images)):
        if i % 1000 == 0: print(i,end=',')
        train_images[i] = u.img_best(train_images[i], size=adj_size, vdif=1, hdif=1)
    for i in range(len(test_images)):
        if i % 1000 == 0: print(i,end=',')
        test_images[i] = u.img_best(test_images[i], size=adj_size, vdif=1, hdif=1)


In [12]:
# 預處理訓練資料
x_train = train_images.reshape((60000, 28 * 28)) #←將 (60000,28,28) 轉換成 (60000,784),餵進DNN之前需要將圖片二階張量展平為一階
x_train = x_train.astype('float32') / 255    #←再將 0~255 的像素值轉換成 0~1 的浮點數, Normalized普適化
x_test = test_images.reshape((10000, 28 * 28))  #}←將 10000 筆測試樣本做同樣的轉換
x_test = x_test.astype('float32') / 255         #}

# 預處理標籤資料
y_train = to_categorical(train_labels)  #←將標籤做 One-hot 編碼
y_test  = to_categorical(test_labels)  #←將測試做 One-hot 編碼
print(y_train.shape), print(y_test.shape)  #(60000, 10) (10000, 10) 原本colume為1,在one-hot之後變10

(60000, 10)
(10000, 10)


(None, None)

In [14]:
#建立多元分類模型
from tensorflow.keras.models import Sequential  #← 匯入 Keras 的序列式模型類別
from tensorflow.keras.layers import Dense       #← 匯入 Keras 的密集層類別

model = Sequential()                 #← 建立序列模型物件
model.add(Dense(512, activation='relu', input_dim= 784)) #← 加入第一層 用Relu避免梯度消失問題,因為神經元不多也不用擔心梯度爆炸。
model.add(Dense(10, activation='softmax'))               #← 加入第二層 多分類輸出用softmax。

model.compile(optimizer='rmsprop',             #← 指定優化器
              loss='categorical_crossentropy', #← 指定損失函數
              metrics=['accuracy'])            #← 指定評量準則

In [16]:
#訓練模型
history = model.fit(x_train, y_train, epochs=5, batch_size=256) #final loss: 0.0088 - accuracy: 0.9978

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0240 - accuracy: 0.9932
Epoch 2/5
60000/60000 [==============================] - 1s 25us/sample - loss: 0.0186 - accuracy: 0.9950
Epoch 3/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.0139 - accuracy: 0.9962
Epoch 4/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0108 - accuracy: 0.9973
Epoch 5/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0088 - accuracy: 0.9978


In [17]:
#評估模型成效
test_loss, test_acc = model.evaluate(x_test, y_test)   #←使用測試樣本及標籤來評估普適能力
print('對測試資料集的準確率：', test_acc)  #output:對測試資料集的準確率： 0.9837

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

對測試資料集的準確率： 0.9837


In [18]:
#程 將模型存檔
model.save('模型_DNN' + fns + '_new.h5')   #← 將模型以指定的檔名存檔